#### Hyperparameter Tuning Using GridSerchCV

Process of choosing optimal paramter for the models is called HPT

Fisrt you can do train test split and train the model with some parameters. But everytime the model is run,the tarain test split changes and we geta different score each time. Instead if we use cross val score in for loop, we have to manually do all the per&comb for different parameters to get the best score which is time consuming if dataset is bigger and no of paramerts are many.So we use GridSearchCV to do the same thing. 

RandomizedSerachCV is another approach which randomly takes values to carry out P&C of parameters. Hence it is efficient than GridSCV. 

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
iris=load_iris()

In [6]:
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['target']=iris.target
df['flower']=df['target'].apply(lambda x:iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [13]:
from sklearn.svm import SVC

In [14]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False)
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00132256, 0.00100412, 0.00100389, 0.00140471, 0.00171633,
        0.00160561]),
 'std_fit_time': array([6.20209600e-04, 9.53388171e-06, 1.01170417e-05, 4.87591500e-04,
        3.89133871e-04, 7.94498592e-04]),
 'mean_score_time': array([0.00088897, 0.00100374, 0.00080171, 0.00122786, 0.00101223,
        0.00152454]),
 'std_score_time': array([1.91074224e-04, 6.50144340e-06, 4.00984941e-04, 3.93116335e-04,
        6.45296956e-04, 7.57625395e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [16]:
cv_result=pd.DataFrame(clf.cv_results_)

In [17]:
cv_result[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [18]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [23]:
clf.best_score_

0.9800000000000001

In [22]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [26]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False,n_iter=2)
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,linear,0.966667
1,10,linear,0.973333


#### How to choose best model for the classification prbm

In [59]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [60]:
# define your model using json file or simple dictionary with required parameters
model_params = {
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'randomclassifier':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
            
        }
        
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    }
    
}

In [61]:
scores=[]
for model_name,mp in model_params.items():
    clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model':model_name,
        'best_params':clf.best_params_,
        'best_score':clf.best_score_
    })

In [62]:
df_best= pd.DataFrame(scores,columns=['model','best_params','best_score'])
df_best
#scores

,model,best_params,best_score
0,svm,"{'C': 1, 'kernel': 'rbf'}",0.980000
1,randomclassifier,{'n_estimators': 5},0.966667
2,logistic_regression,{'C': 5},0.966667


#### Excercise

In [63]:
from sklearn.datasets import load_digits
digits=load_digits()

In [64]:
df=pd.DataFrame(digits.data,digits.target,columns=digits.feature_names)
df.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [65]:
#Hyperparamter tuning with best model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier


In [68]:
model_params = {
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear','poly','sigmoid']
        }
    },
    'randomclassifier':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10,20]
            
        }
        
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    },
     'GaussianNB':{
        'model':GaussianNB(),
        'params':{
            
        }
    },
    'MultinomialNB':{
        'model':MultinomialNB(),
        'params':{
            'alpha':[15,5,10]
        }
    },
    
    'decisiontree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy', 'log_loss'],
            'splitter':["best", "random"]
            
        }
        
    }
    
} 

In [69]:
scores=[]
for model_name,mp in model_params.items():
    clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(digits.data,digits.target)
    scores.append({
        'model':model_name,
        'best_params':clf.best_params_,
        'best_score':clf.best_score_
    })
df_bestmodel= pd.DataFrame(scores,columns=['model','best_params','best_score'])
df_bestmodel

,model,best_params,best_score
0,svm,"{'C': 1, 'kernel': 'poly'}",0.968842
1,randomclassifier,{'n_estimators': 20},0.923788
2,logistic_regression,{'C': 1},0.922114
3,GaussianNB,{},0.806928
4,MultinomialNB,{'alpha': 10},0.874246
5,decisiontree,"{'criterion': 'log_loss', 'splitter': 'best'}",0.820286


Here, SVM model gives the best score